In [1]:
import numpy as np
import zarr

from faim_hcs.Zarr import write_labels_to_group
from faim_hcs.mobie import add_labels_view, compute_aggregate_table_values
from mobie.metadata import read_dataset_metadata
from mobie.validation import validate_dataset
from ome_zarr.io import parse_url
from pathlib import Path
from skimage.measure import label
from tqdm import tqdm


## Segment projection image in plate

In [2]:
storage = parse_url(path="./zarr-files/Projection-Mix.zarr/", mode="w").store
plate = zarr.group(store=storage)

In [3]:
wells = plate.attrs.asdict()["plate"]["wells"]
wells

[{'columnIndex': 7, 'path': 'E/8', 'rowIndex': 4},
 {'columnIndex': 6, 'path': 'E/7', 'rowIndex': 4}]

In [4]:
threshold = 20000

In [5]:
for well in tqdm(wells):
    group = plate[well["path"]][0]["projections"]
    img = group[0]
    labels = label(img[0]>threshold)[np.newaxis, :]  # simple threshold segmentation
    write_labels_to_group(labels=labels, labels_name="simple_threshold", parent_group=group)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 25.59it/s]


In [6]:
plate['E/7/0/projections/labels/simple_threshold/0'].info

Name,/E/7/0/projections/labels/simple_threshold/0
Type,zarr.core.Array
Data type,int32
Shape,"(1, 512, 1024)"
Chunk shape,"(1, 512, 1024)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.FSStore
No. bytes,2097152 (2.0M)
No. bytes stored,395


## Add segmentation to MoBIE project

In [7]:
mobie_project_folder = "./mobie-example-project"
dataset_name = "Projection-Mix"
dataset_folder = Path(mobie_project_folder, dataset_name)

In [8]:
dataset_metadata = read_dataset_metadata(dataset_folder=dataset_folder)

In [9]:
from IPython.display import JSON
JSON(dataset_metadata)

<IPython.core.display.JSON object>

In [10]:
view = dataset_metadata["views"]["Projections"]
JSON(view)

<IPython.core.display.JSON object>

In [11]:
add_labels_view(
    plate=plate,
    dataset_folder=dataset_folder,
    well_group="0/projections",
    channel=0,
    label_name="simple_threshold",
    view_name="Projections",
    extra_properties=("area", "bbox")
)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

/tungstenfs/nobackup/gmicro/eglijan/miniconda3/envs/ggrossha/lib/python3.9/site-packages/mobie/metadata/dataset_metadata.py:72: UserWarning: A view with name Projections already exists for the dataset mobie-example-project/Projection-Mix
  warnings.warn(msg)


In [12]:
dataset_metadata = read_dataset_metadata(dataset_folder=dataset_folder)
view = dataset_metadata["views"]["Projections"]
JSON(view)

<IPython.core.display.JSON object>

In [13]:
# Validate MoBIE dataset integrety
validate_dataset(dataset_folder=dataset_folder)

Check views for dataset Projection-Mix: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 48.06it/s]
Check view files for dataset Projection-Mix: 0it [00:00, ?it/s]


## Aggregate tables into common wells table

In [14]:
compute_aggregate_table_values(
    dataset_folder=dataset_folder,
    table_suffix="simple_threshold",
    aggregation_dict={"area": ["mean", "min", "max", "median"]},
)


           area_mean_simple_threshold  area_min_simple_threshold  \
region_id                                                          
E07                        100.633492                   1.868142   
E08                        350.943864                   1.868142   

           area_max_simple_threshold  area_median_simple_threshold  
region_id                                                           
E07                       128.901815                    106.484108  
E08                      1081.654357                    218.572642  
